<a href="https://colab.research.google.com/github/nyongja/basic-of-reinforcement-learning/blob/main/Vanilla_Policy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gym

In [2]:
!pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 51kB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 491kB 32.0MB/s 
     |████████████████████████████████| 3.2MB 41.4MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


## 기본적인 정책 경사 에이전트 구현

2개 이상의 액션에도 일반화가 가능

In [3]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import matplotlib.pyplot as plta

In [4]:
%matplotlib inline

오픈AI 짐에서 특정 환경을 불러오는 코드는 다음과 같음

In [6]:
env = gym.make('CartPole-v0')

## 정책 기반 에이전트
다음은 보상 함수 및 에이전트를 구현하는 코드

In [7]:
gamma = 0.99

def discount_rewards(r) :
  # 보상의 1D 실수 배열을 취해서 할인된 보상을 계산
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)) :
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

In [18]:
class agent() :
  def __init__(self, lr, s_size, a_size, h_size) :
    # 네트워크의 피드포워드 부분. 에이전트는 상태를 받아 액션을 출력
    self.state_in = tf.placeholder(shape = [None, s_size], dtype = tf.float32)
    hidden = slim.fully_connected(self.state_in, h_size, biases_initializer = None,
                                  activation_fn = tf.nn.relu)
    self.output = slim.fully_connected(hidden, a_size, biases_initializer = None,
                                       activation_fn = tf.nn.softmax)
    self.chosen_action = tf.argmax(self.output, 1)

    # 학습 과정 구현. 비용을 계산하기 위해 보상과 액션을 네트워크에 피드하고,
    # 네트워크를 업데이트하는 데에 이를 이용함.
    self.reward_holder = tf.placeholder(shape = [None], dtype = tf.float32)
    self.action_holder = tf.placeholder(shape = [None], dtype = tf.int32)

    self.indexes = tf.range(0, tf.shape(self.output)[0]) * tf.shape(self.output)[1] + self.action_holder
    self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), self.indexes)

    self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs) * self.reward_holder)

    tvars = tf.trainable_variables()
    self.gradient_holders = []
    for idx, var in enumerate(tvars) :
      placeholder = tf.placeholder(tf.float32, name = str(idx) + '_holder')
      self.gradient_holders.append(placeholder)

    self.gradients = tf.gradients(self.loss, tvars)

    optimizer = tf.train.AdamOptimizer(learning_rate = lr)
    self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders, tvars))

## 에이전트 학습시키기

마지막으로 학습 과정을 구현한 코드.  
큰 구조는 앞 장 예제들과 같음.

In [21]:
tf.reset_default_graph() #Clear the Tensorflow graph.

myAgent = agent(lr=1e-2,s_size=4,a_size=2,h_size=8) #Load the agent.

total_episodes = 5000 #Set total number of episodes to train agent on.
max_ep = 999
update_frequency = 5

init = tf.global_variables_initializer()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    total_reward = []
    total_length = []
        
    gradBuffer = sess.run(tf.trainable_variables())
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
        
    while i < total_episodes:
        s = env.reset()
        running_reward = 0
        ep_history = []
        for j in range(max_ep):
            #Probabilistically pick an action given our network outputs.
            a_dist = sess.run(myAgent.output,feed_dict={myAgent.state_in:[s]})
            a = np.random.choice(a_dist[0],p=a_dist[0])
            a = np.argmax(a_dist == a)

            s1,r,d,_ = env.step(a) #Get our reward for taking an action given a bandit.
            ep_history.append([s,a,r,s1])
            s = s1
            running_reward += r
            if d == True:
                #Update the network.
                ep_history = np.array(ep_history)
                ep_history[:,2] = discount_rewards(ep_history[:,2])
                feed_dict={myAgent.reward_holder:ep_history[:,2],
                        myAgent.action_holder:ep_history[:,1],myAgent.state_in:np.vstack(ep_history[:,0])}
                grads = sess.run(myAgent.gradients, feed_dict=feed_dict)
                for idx,grad in enumerate(grads):
                    gradBuffer[idx] += grad

                if i % update_frequency == 0 and i != 0:
                    feed_dict= dictionary = dict(zip(myAgent.gradient_holders, gradBuffer))
                    _ = sess.run(myAgent.update_batch, feed_dict=feed_dict)
                    for ix,grad in enumerate(gradBuffer):
                        gradBuffer[ix] = grad * 0
                
                total_reward.append(running_reward)
                total_length.append(j)
                break

        
            #Update our running tally of scores.
        if i % 100 == 0:
            print(np.mean(total_reward[-100:]))
        i += 1

20.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


29.1
33.27
36.44
44.76
78.14
120.03
152.7
166.83
174.41
184.61
174.32
179.57
189.24
195.46
194.92
196.9
196.49
196.37
198.17
198.44
199.27
200.0
199.09
198.77
200.0
200.0
200.0
200.0
200.0
200.0
198.06
198.39
193.64
179.38
179.5
197.54
200.0
199.87
199.6
200.0
200.0
200.0
199.57
195.9
172.99
179.61
186.81
174.57
183.38
